# Povezivanje sa drive-om i git-om



In [3]:
#spajam ga sa mojim drajvom gde su svi fajlovi
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
#tačna lokacija
%cd "drive/My Drive/neural_style_transfer_audio"

/content/drive/My Drive/neural_style_transfer_audio


In [ ]:
!git clone https://github.com/Emilija2000/PSIML6_Voice_style_transfer.git

In [5]:
%cd PSIML6_Voice_style_transfer/

/content/drive/My Drive/neural_style_transfer_audio/PSIML6_Voice_style_transfer


In [6]:
!git pull

Already up to date.


#Import svih potrebnih stvari

In [ ]:
!pip install pydub
!pip install torchaudio
!pip install scipy

     |████████████████████████████████| 6.7MB 3.6MB/s 


In [ ]:
#ubacivanje nasih fajlova kao biblioteka
import dataloader

In [ ]:
dataset = dataloader.AccentDataset('data/recordings/recordings','data/speakers_all.csv')
print (dataset.__len__())
vect,label = dataset.__getitem__(1)

1154


NameError: ignored

#Prekopiran ceo kod - isprobavanje

In [ ]:
from collections import defaultdict
import copy
import csv
import matplotlib.pyplot as plt
import numpy as np
import os
from os import path
from pydub import AudioSegment
import scipy
import scipy.signal
from scipy.io import wavfile
import time
import torch
import torchvision
import librosa                    
import librosa.display


AudioSegment.converter = "ffmpeg"
allowed_languages = ["arabic","dutch","french","german","korean","mandarin","russian"]
mean = 280.40387

def readAudio(mp3_path):
    # files             
    #dst = mp3_path[:-4] + ".wav"

    # convert mp3 to wav   
    #if not os.path.exists(dst):                                                         
        #sound = AudioSegment.from_mp3(mp3_path)
        #sound.export(dst, format="wav")

    # read wav and convert to spectrogram
    #sr, data = wavfile.read(dst)
    #plt.plot(data)
    
    #nperseg = int(sr * 0.001 * 20)
    #frequencies, times, spectrogram = scipy.signal.spectrogram(data, sr, nperseg=nperseg, window=scipy.signal.hann(nperseg))

    y, sr = librosa.load(mp3_path)
    melSpec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    melSpec_dB = librosa.power_to_db(melSpec, ref=np.max)
    #print(melSpec_dB)
    return melSpec_dB, sr

class AccentDataset(torch.utils.data.Dataset):
    def __init__(self, path, csv_path):
        self.data_names = []
        self.classes = []
        self.encod_data = []

        with open(csv_path) as csvfile:
            csv_reader = csv.reader(csvfile,delimiter=',')
            for row in csv_reader:
                if (row[8] == "FALSE" and row[4] in allowed_languages):
                    mp3_path = os.path.join(path,row[3]+'.mp3')
                    self.data_names.append((mp3_path, row[4]))
                    if (row[4] not in self.classes):
                        self.classes.append(row[4])

            for data in self.data_names:
                label = torch.zeros(len(self.classes)) 
                label[self.classes.index(data[1])] = 1
                self.encod_data.append((data[0],label))

    def __getitem__(self,index):
        dat, sr = readAudio(self.data_names[index][0])
        #print(dat)
        label_onehot = self.encod_data[index][1]
        label_onehot = torch.Tensor(label_onehot)
        #label_onehot = label.type(torch.long)
        #label_onehot = label.reshape(len(label),1,1)
        label = torch.argmax(label_onehot).item()
        #label = label.reshape(1,1)

        dat = torch.Tensor(dat).reshape(1,dat.shape[0],dat.shape[1])

        return dat, label

    def __len__(self):
        return len(self.data_names)
    

In [ ]:
#isprobavanje
dataset = AccentDataset('data/recordings/recordings','data/speakers_all.csv')
print (dataset.__len__())
dat,label = dataset.__getitem__(1)
print(dat.size())


413
torch.Size([1, 128, 1601])


In [ ]:
print(label.shape)

#Model klasifikatora


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Classifier(nn.Module):
    def __init__(self, output_num):
        super(Classifier,self).__init__()
        modules = []

        modules.append(nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = 2, stride = 2))
        modules.append(nn.BatchNorm2d(8))
        modules.append(nn.ReLU(inplace = True))

        modules.append(nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 2, stride = 2))
        modules.append(nn.BatchNorm2d(16))
        modules.append(nn.ReLU(inplace = True))

        modules.append(nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 2, stride = 2))
        modules.append(nn.BatchNorm2d(32))
        modules.append(nn.ReLU(inplace = True))

        modules.append(nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 2, stride = 2))
        modules.append(nn.BatchNorm2d(64))
        modules.append(nn.ReLU(inplace = True))

        modules.append(nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 2, stride = 2))
        modules.append(nn.BatchNorm2d(128))
        modules.append(nn.ReLU(inplace = True))

        modules.append(nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 2, stride = 2))
        modules.append(nn.BatchNorm2d(256))
        modules.append(nn.ReLU(inplace = True))

        modules.append(nn.AdaptiveAvgPool2d(output_size = 1))


        self.fc=nn.Linear(in_features=256,out_features=output_num,bias=False)

        #modules.append(nn.Conv2d(in_channels = 64, out_channels = 32, kernel_size = (3,4), stride = (1,2)))
        #modules.append(nn.BatchNorm2d(32))
        #modules.append(nn.ReLU(inplace = True))

        #modules.append(nn.Conv2d(in_channels = 32, out_channels = 16, kernel_size = (3,4), stride = (1,2)))
        #modules.append(nn.BatchNorm2d(16))
        #modules.append(nn.ReLU(inplace = True))

        #modules.append(nn.Conv2d(in_channels = 16, out_channels = output_num, kernel_size = (1,4), stride = (1,2)))

        #modules.append(nn.Softmax())

        self.sequence = nn.Sequential(*modules)


    def forward(self,x):
        y = self.sequence(x)
        y=y.flatten()
        return self.fc(y)

In [ ]:
Classif = Classifier(8)

#Trening klasifikatora

In [ ]:
learning_rate = 0.001
optimizer = torch.optim.Adam(Classif.parameters(), lr = learning_rate)

'''def cross_entropy_one_hot(output, target):
    #target = torch.reshape(target,(1,target.size(0)))
    _, labels = target.max(dim=1)
    res = nn.CrossEntropyLoss()(output, labels)
    print(res)
    return res'''

loss_func = nn.CrossEntropyLoss()

In [ ]:
# make train and valid splits
import random
random.seed(0)  # rng seed, set to 0 for reproducibility

dataset_indices = list(range(len(dataset)))
random.shuffle(dataset_indices)  # shuffle the indices before splitting (use random.shuffle)

#split datasets
train_split_indices = dataset_indices[:int(len(dataset_indices)*0.6)]  # get the training split indices
valid_split_indices = dataset_indices[int(len(dataset_indices)*0.6):int(len(dataset_indices)*0.8)]  # get the validation split indices 
test_split_indices = dataset_indices[int(len(dataset_indices)*0.8):]

s=np.zeros(8)

for i in range(dataset.__len__()):
    s[dataset.__getitem__(i)[3]]+=1
#print("nesto")
weights = []
for i in range(dataset.__len__()):
    weights.append(dataset.__len__()/s[dataset.__getitem__(i)[3]])

train_subset_sampler = torch.utils.data.SubsetRandomSampler(train_split_indices)
valid_subset_sampler = torch.utils.data.SubsetRandomSampler(valid_split_indices)
test_subset_sampler = torch.utils.data.SubsetRandomSampler(test_split_indices)

all_sampler = torch.utils.data.WeightedRandomSampler(weights,dataset.__len__(),False)

In [ ]:
print(weights)

[4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607843137, 4.049019607

In [ ]:
batch_size = 1
num_workers=0

train_dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, sampler=train_subset_sampler, num_workers=num_workers, drop_last=True)
valid_dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, sampler=valid_subset_sampler, num_workers=num_workers, drop_last=True)
test_dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=test_split_indices, num_workers=num_workers, drop_last=True)

all_dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, sampler=all_sampler, num_workers=num_workers, drop_last=True)

In [ ]:
device = torch.device('cuda')
Classif = Classif.cuda()

In [ ]:
num_epochs = 200

for epoch in range(num_epochs):
    Classif.train()
    
    for i, (spect,label) in enumerate(train_dataloader):
    #for i in range(200):
        #f,t,spect,label = dataset.__getitem__(200+i)
        #spect = spect.reshape(1,spect.size(0),spect.size(1),spect.size(2))
        #label = label.reshape(1,label.size(0),label.size(1))
        spect = spect.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        output = Classif(spect)
        output=output.reshape(1,-1) 
        #print("output=",output," label=",label)
        loss = loss_func(output, label)

        loss.backward()
        optimizer.step()

        if (i+1) % (10) == 0:
          print(f'epoch: {epoch}   iter: {i+1}   batch_loss: {loss}')
            #print('output: ', output,' label: ',label)

    Classif.eval()
    
    correct = 0
    total = 0

    with torch.no_grad():
        for (spect,label) in valid_dataloader:
        #for i in range(200):
            #f,t,spect,label = dataset.__getitem__(200+i)
            #spect = spect.reshape(1,spect.size(0),spect.size(1),spect.size(2))
            #label = label.reshape(1,label.size(0),label.size(1))
            spect = spect.to(device)
            label = label.to(device)

            output = Classif(spect)

            predicted = output.reshape(8)
            real_class = label
            predicted = torch.argmax(predicted)
            #print("predicted=",predicted," real_class=",real_class)
            correct += np.sum(int(predicted==real_class))
            total += label.size(0)
    
    #for name, param in Classif.named_parameters():
    #  if param.requires_grad:
    #    print (name, param.data)

    validation_accuracy = 100 * float(correct)/total
    print(f'epoch: {epoch}   validation accuracy: {validation_accuracy}%' )


        
        

epoch: 0   iter: 10   batch_loss: 1.961584210395813
epoch: 0   iter: 20   batch_loss: 1.9179778099060059
epoch: 0   iter: 30   batch_loss: 2.6359667778015137
epoch: 0   iter: 40   batch_loss: 2.34479022026062
epoch: 0   iter: 50   batch_loss: 1.3518435955047607
epoch: 0   iter: 60   batch_loss: 2.300403594970703
epoch: 0   iter: 70   batch_loss: 1.863419532775879
epoch: 0   iter: 80   batch_loss: 1.9057613611221313
epoch: 0   iter: 90   batch_loss: 2.3837828636169434
epoch: 0   iter: 100   batch_loss: 2.2802505493164062
epoch: 0   iter: 110   batch_loss: 1.1071181297302246
epoch: 0   iter: 120   batch_loss: 1.9339103698730469
epoch: 0   iter: 130   batch_loss: 1.3386623859405518
epoch: 0   iter: 140   batch_loss: 1.7295008897781372
epoch: 0   iter: 150   batch_loss: 2.8083724975585938
epoch: 0   iter: 160   batch_loss: 2.4678707122802734
epoch: 0   iter: 170   batch_loss: 2.9179444313049316
epoch: 0   iter: 180   batch_loss: 1.6405963897705078
epoch: 0   iter: 190   batch_loss: 2.27748

KeyboardInterrupt: ignored

In [ ]:
for (f,t,spect,label) in valid_dataloader:
    print(spect)
    break

tensor([[[[2.6051e-02, 5.2489e-04, 3.0030e-04,  ..., 3.6292e-02,
           1.6559e-03, 3.8184e-02],
          [5.9030e-04, 8.9344e-04, 2.8802e-03,  ..., 9.4530e-03,
           3.5228e-03, 2.2362e-02],
          [1.5108e-01, 2.7274e-03, 7.0989e-02,  ..., 5.2879e-01,
           2.1252e-01, 1.4154e-01],
          ...,
          [1.6472e-06, 3.8050e-06, 1.1337e-06,  ..., 2.7850e-06,
           1.3825e-06, 9.6127e-06],
          [4.0250e-06, 3.2164e-05, 4.2384e-07,  ..., 2.1258e-06,
           4.4186e-06, 2.1877e-05],
          [6.9165e-08, 1.9403e-05, 4.3224e-07,  ..., 3.2855e-06,
           1.8415e-06, 1.0861e-05]]]])


In [ ]:
label.size()

torch.Size([1, 8, 1, 1])